In [ ]:
# some utilities
import os
import numpy as np
from util import base64_to_pil

# Flask
from flask import Flask, redirect, url_for, request, render_template, Response, jsonify, redirect

#tensorflow
import tensorflow as tf
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2


# Variables 
# Change them if you are using custom model or pretrained model with saved weigths
Model_json = "ImageClassifier.json"
Model_weigths = "ImageClassifier.h5"


# Declare a flask app
app = Flask(__name__)

def get_ImageClassifierModel():
    #model = MobileNetV2(weights='imagenet')

    # Loading the pretrained model
    model_json = open("ImageClassifier.json", 'r')
    loaded_model_json = model_json.read()
    model_json.close()
    model = tf.keras.models.model_from_json(loaded_model_json)
    model.load_weights("ImageClassifier.h5")

    return model  
    


def model_predict(img, model):
    '''
    Prediction Function for model.
    Arguments: 
        img: is address to image
        model : image classification model
    '''
    img = img.resize((180, 180))

    # Preprocessing the image
    x = tf.keras.preprocessing.image.img_to_array(img)
    # x = np.true_divide(x, 255)
    x = tf.expand_dims(x, axis=0)

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
    #x = preprocess_input(x, mode='tf')

    preds = model.predict(x)
    return preds


@app.route('/', methods=['GET'])
def index():
    '''
    Render the main page
    '''
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def predict():
    '''
    predict function to predict the image
    Api hits this function when someone clicks submit.
    '''
    if request.method == 'POST':
        # Get the image from post request
        img = base64_to_pil(request.json)
        
        # initialize model
        model = get_ImageClassifierModel()

        # Make prediction
        preds = model_predict(img, model)

        pred_proba = "{:.3f}".format(np.argmax(preds))    # Max probability
        pred_class = ['angulate','circle','egg','long','triangle']
        # pred_class = decode_predictions(preds, top=1)   # ImageNet Decode
        print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(pred_class[np.argmax(preds[0])], 100 * np.max(preds[0]))
        )
        
        result = np.argmax(preds[0])
        result = pred_class[result]
        # result = str(pred_class[0][0][1])               # Convert to string
        # result = result.replace('_', ' ').capitalize()
        
        # Serialize the result, you can add additional fields
        return jsonify(result=result, probability=pred_proba)
    return None


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Mar/2022 17:47:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2022 17:47:49] "GET /static/main.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 17:47:49] "GET /static/main.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Mar/2022 17:47:56] "POST /predict HTTP/1.1" 200 -


This image most likely belongs to circle with a 80.76 percent confidence.


127.0.0.1 - - [11/Mar/2022 17:49:23] "POST /predict HTTP/1.1" 200 -


This image most likely belongs to egg with a 67.35 percent confidence.
